In [3]:
import pandas as pd
import numpy as np
import math
import glob
import os

In [35]:
wd = os.getcwd()
files = glob.glob(wd + '/*.xlsx')
for f in files:
    if "US Open" in f:
        files.remove(f)

In [49]:
def probability(elo_1, elo_2):
    
    return 1/(1 + math.pow(10, (elo_1 - elo_2)/400))

In [50]:
def elo_rating(rating_player, rating_opponent, K):
    
    prob_a = probability(rating_opponent, rating_player)
    prob_b = probability(rating_player, rating_opponent)
    
    rating_player = rating_player + K * (1 - prob_a)
    rating_opponent = rating_opponent + K * (0 - prob_b)
    
    return rating_player, rating_opponent

In [123]:
def apply_elo(df):

    for i in range(df.shape[0]):
        elo_winner, elo_loser = elo_rating(df.at[i, 'winner_elo'], df.at[i, 'loser_elo'], 32)

        df.at[i, 'winner_elo'] = np.around(elo_winner)
        df.at[i, 'loser_elo'] = np.around(elo_loser)

        winner_name = df.winner_name[i]
        loser_name = df.loser_name[i]

        sub_ww = df[df['winner_name']==winner_name].winner_elo.iloc[i+1]
        sub_ll = df[df['loser_name']==loser_name].loser_elo.iloc[i+1]
        sub_lw = df[df['loser_name']==winner_name].loser_elo.iloc[i+1]
        sub_wl = df[df['winner_name']==loser_name].winner_elo.iloc[i+1]

        #sub_ww.replace(to_replace=sub_ww.values.tolist(), value=np.around(elo_winner), inplace=True)
        #sub_ll.replace(to_replace=sub_ll.values.tolist(), value=np.around(elo_loser), inplace=True)
        #sub_lw.replace(to_replace=sub_lw.values.tolist(), value=np.around(elo_winner), inplace=True)
        #sub_wl.replace(to_replace=sub_wl.values.tolist(), value=np.around(elo_loser), inplace=True)
        
        df[df['winner_name']==winner_name].winner_elo.iloc[i+1] = np.around(elo_winner)
        df[df['loser_name']==loser_name].loser_elo.iloc[i+1] = np.around(elo_loser)
        df[df['loser_name']==winner_name].loser_elo.iloc[i+1] = np.around(elo_winner)
        df[df['winner_name']==loser_name].winner_elo.iloc[i+1] = np.around(elo_loser)

        #df.update(sub_ww)
        #df.update(sub_ll)
        #df.update(sub_lw)
        #df.update(sub_wl)
        
    return df

In [112]:
sub_ww = df0[df0['winner_name']=='Marcos Baghdatis'].winner_elo.iloc[1:]
sub_ll = df0[df0['loser_name']=='Mardy Fish'].loser_elo.iloc[1:]
sub_lw = df0[df0['loser_name']=='Marcos Baghdatis'].loser_elo.iloc[1:]
sub_wl = df0[df0['winner_name']=='Mardy Fish'].winner_elo.iloc[1:]


In [114]:
sub_ll

98      1469.0
184     1469.0
507     1469.0
588     1469.0
653     1469.0
770     1469.0
1178    1469.0
1318    1469.0
1352    1469.0
1575    1469.0
1871    1469.0
1977    1469.0
2104    1469.0
2350    1469.0
Name: loser_elo, dtype: float64

In [124]:
df0 = pd.read_excel(files[0])
df0.tourney_date = pd.to_datetime(df0.tourney_date, yearfirst=True, format='%Y%m%d')
df0 = df0.sort_values(by='tourney_date').reset_index(drop=True)

data = np.ones((len(df0), 1))*1500
df0['winner_elo'] = data
df0['loser_elo'] = data

df0 = apply_elo(df0)

C:\Users\PC\AppData\Local\Temp\ipykernel_11080\2768292974.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['winner_name']==winner_name].winner_elo.iloc[i+1] = np.around(elo_winner)
C:\Users\PC\AppData\Local\Temp\ipykernel_11080\2768292974.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['loser_name']==loser_name].loser_elo.iloc[i+1] = np.around(elo_loser)
C:\Users\PC\AppData\Local\Temp\ipykernel_11080\2768292974.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

IndexError: single positional indexer is out-of-bounds

In [86]:
df0

,tourney_id,tourney_name,series,court,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,winner_elo,loser_elo
0,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,6,104571,...,1.377,3.410,1.400,2.750,NaN,NaN,NaN,NaN,1516.0,1484.0
1,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,30,103285,...,2.350,1.685,2.000,1.727,NaN,NaN,NaN,NaN,1516.0,1484.0
2,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,16,103285,...,1.087,9.950,1.030,10.000,NaN,NaN,NaN,NaN,1516.0,1484.0
3,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,24,103285,...,1.159,6.390,1.125,5.500,NaN,NaN,NaN,NaN,1531.0,1485.0
4,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,11,103722,...,1.483,2.900,1.440,2.630,NaN,NaN,NaN,NaN,1516.0,1484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,1,103819,...,1.140,7.100,1.130,6.000,1.14,8.20,1.11,6.39,1531.0,1485.0
2675,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,4,103819,...,1.420,3.180,1.360,3.000,1.50,3.24,1.40,2.92,1531.0,1485.0
2676,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,14,104925,...,1.480,2.920,1.440,2.750,1.48,3.15,1.40,2.88,1517.0,1499.0
2677,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,13,104925,...,1.250,4.600,1.250,3.750,1.30,4.76,1.24,4.03,1515.0,1469.0


In [107]:
df0[df0['winner_name']=='Marcos Baghdatis'].winner_elo.iloc[:]

0       1516.0
97      1531.0
98      1531.0
101     1531.0
102     1531.0
117     1531.0
264     1531.0
269     1531.0
442     1531.0
500     1531.0
599     1531.0
601     1531.0
602     1531.0
674     1531.0
675     1531.0
820     1531.0
1072    1531.0
1073    1531.0
1074    1531.0
1151    1531.0
1192    1531.0
1193    1531.0
1238    1531.0
1239    1531.0
1851    1531.0
1870    1531.0
1878    1515.0
1879    1515.0
1880    1515.0
1999    1515.0
2012    1515.0
2013    1514.0
2016    1515.0
2045    1515.0
2046    1515.0
2277    1515.0
2286    1515.0
2436    1515.0
2437    1515.0
2438    1514.0
2482    1515.0
Name: winner_elo, dtype: float64

In [120]:
df0[['winner_name', 'loser_name', 'winner_elo', 'loser_elo']].head(51)

,winner_name,loser_name,winner_elo,loser_elo
0,Marcos Baghdatis,Mardy Fish,1516.0,1484.0
1,Radek Stepanek,Gael Monfils,1516.0,1484.0
2,Radek Stepanek,John Millman,1531.0,1485.0
3,Radek Stepanek,Alexandr Dolgopolov,1546.0,1485.0
4,Florent Serra,Julian Reister,1516.0,1484.0
5,Andy Roddick,Radek Stepanek,1516.0,1484.0
6,Andy Roddick,Peter Luczak,1516.0,1484.0
7,Andy Roddick,Richard Gasquet,1516.0,1484.0
8,Andy Roddick,Tomas Berdych,1516.0,1484.0
9,Andy Roddick,Carsten Ball,1516.0,1484.0


In [74]:
sub_ww = df0[df0['loser_name']=='Marcos Baghdatis'][['winner_name', 'loser_name', 'winner_elo', 'loser_elo']]
sub_ww

,winner_name,loser_name,winner_elo,loser_elo
26,Tomas Berdych,Marcos Baghdatis,1852.0,1692.0
155,Lleyton Hewitt,Marcos Baghdatis,1868.0,1836.0
452,Nikolay Davydenko,Marcos Baghdatis,1868.0,1836.0
506,Michael Llodra,Marcos Baghdatis,1868.0,1836.0
630,Novak Djokovic,Marcos Baghdatis,1868.0,1836.0
713,Tommy Robredo,Marcos Baghdatis,1868.0,1836.0
796,Marin Cilic,Marcos Baghdatis,1868.0,1836.0
887,Albert Montanes,Marcos Baghdatis,1868.0,1836.0
1029,Ernests Gulbis,Marcos Baghdatis,1868.0,1836.0
1067,Marin Cilic,Marcos Baghdatis,1868.0,1836.0


In [104]:
sub_ww = df0[((df0['winner_name']=='Marcos Baghdatis') | (df0['loser_name']=='Marcos Baghdatis'))][['winner_name', 'loser_name', 'winner_elo', 'loser_elo']]
sub_ww.head(51)

,winner_name,loser_name,winner_elo,loser_elo
0,Marcos Baghdatis,Mardy Fish,1516.0,1484.0
26,Tomas Berdych,Marcos Baghdatis,1517.0,1499.0
97,Marcos Baghdatis,Richard Gasquet,1531.0,1485.0
98,Marcos Baghdatis,Mardy Fish,1531.0,1469.0
101,Marcos Baghdatis,Lleyton Hewitt,1531.0,1485.0
102,Marcos Baghdatis,Nick Lindahl,1531.0,1485.0
117,Marcos Baghdatis,Viktor Troicki,1531.0,1485.0
155,Lleyton Hewitt,Marcos Baghdatis,1517.0,1499.0
264,Marcos Baghdatis,Paolo Lorenzi,1531.0,1485.0
269,Marcos Baghdatis,David Ferrer,1531.0,1485.0


In [90]:
elow, elol = elo_rating(1500, 1500, 32)
print(elow, elol)

1516.0 1484.0


In [66]:
df0

,tourney_id,tourney_name,series,court,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,winner_elo,loser_elo
0,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,6,104571,...,1.377,3.410,1.400,2.750,NaN,NaN,NaN,NaN,1515.0000,1485.0000
1,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,30,103285,...,2.350,1.685,2.000,1.727,NaN,NaN,NaN,NaN,1515.0000,1485.0000
2,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,16,103285,...,1.087,9.950,1.030,10.000,NaN,NaN,NaN,NaN,1515.0000,1485.0000
3,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,24,103285,...,1.159,6.390,1.125,5.500,NaN,NaN,NaN,NaN,1529.3528,1485.6472
4,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,11,103722,...,1.483,2.900,1.440,2.630,NaN,NaN,NaN,NaN,1515.0000,1485.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,1,103819,...,1.140,7.100,1.130,6.000,1.14,8.20,1.11,6.39,1529.3528,1485.6472
2675,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,4,103819,...,1.420,3.180,1.360,3.000,1.50,3.24,1.40,2.92,1529.3528,1485.6472
2676,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,14,104925,...,1.480,2.920,1.440,2.750,1.48,3.15,1.40,2.88,1515.6472,1499.3528
2677,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,13,104925,...,1.250,4.600,1.250,3.750,1.30,4.76,1.24,4.03,1514.3528,1470.6472


In [47]:
ini = 1500
for f in files:
    ini = 1600

In [48]:
ini

1600

In [39]:
df1 = pd.read_excel(files[0])
df2 = pd.read_excel(files[1])

df1_cols = df1.columns
df2_cols = df2.columns

In [42]:
len(df2_cols)

78

In [38]:
df1.equals(df2)

False

In [46]:
init_elo = 1500

for i in files:

    df = pd.read_excel(i)
    df.tourney_date = pd.to_datetime(df.tourney_date, yearfirst=True, format='%Y%m%d')
    df = df.sort_values(by='tourney_date').reset_index(drop=True)
    
    data = np.ones((len(df), 1))*init_elo
    df['winner_elo'] = data
    df['loser_elo'] = data


C:\Users\PC\Desktop\Hasl\Rahber Tariani\2010_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2011_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2012_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2013_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2014_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2015_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2016_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2017_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2018_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2019_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2020_mens.xlsx
C:\Users\PC\Desktop\Hasl\Rahber Tariani\2021_mens.xlsx


In [313]:
df.tourney_date = pd.to_datetime(df.tourney_date, yearfirst=True, format='%Y%m%d')
df = df.sort_values(by='tourney_date').reset_index(drop=True)

In [314]:
df.shape

(2679, 78)

In [315]:
df

,tourney_id,tourney_name,series,court,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,6,104571,...,1.40,2.75,1.377,3.410,1.400,2.750,NaN,NaN,NaN,NaN
1,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,30,103285,...,2.10,1.67,2.350,1.685,2.000,1.727,NaN,NaN,NaN,NaN
2,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,16,103285,...,1.04,9.00,1.087,9.950,1.030,10.000,NaN,NaN,NaN,NaN
3,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,24,103285,...,1.12,5.50,1.159,6.390,1.125,5.500,NaN,NaN,NaN,NaN
4,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,11,103722,...,1.44,2.62,1.483,2.900,1.440,2.630,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,1,103819,...,1.13,5.50,1.140,7.100,1.130,6.000,1.14,8.20,1.11,6.39
2675,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,4,103819,...,1.44,2.63,1.420,3.180,1.360,3.000,1.50,3.24,1.40,2.92
2676,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,14,104925,...,1.36,3.00,1.480,2.920,1.440,2.750,1.48,3.15,1.40,2.88
2677,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,13,104925,...,1.25,3.75,1.250,4.600,1.250,3.750,1.30,4.76,1.24,4.03


In [316]:
data = np.ones((len(df), 1))*1500

df['winner_elo'] = data
df['loser_elo'] = data

In [317]:
def probability(elo_1, elo_2):
    
    return 1/(1 + math.pow(10, (elo_1 - elo_2)/400))

In [318]:
def elo_rating(rating_player, rating_opponent, K):
    
    prob_a = probability(rating_opponent, rating_player)
    prob_b = probability(rating_player, rating_opponent)
    
    rating_player = rating_player + K * (1 - prob_a)
    rating_opponent = rating_opponent + K * (0 - prob_b)
    
    return rating_player, rating_opponent

In [319]:
for i in range(df.shape[0]):
    elo_winner, elo_loser = elo_rating(df.at[i, 'winner_elo'], df.at[i, 'loser_elo'], 30)
    
    df.at[i, 'winner_elo'] = elo_winner
    df.at[i, 'loser_elo'] = elo_loser
    
    winner_name = df.winner_name[i]
    loser_name = df.loser_name[i]
    
    sub_ww = df[df['winner_name']==winner_name].winner_elo.iloc[i+1:]
    sub_ll = df[df['loser_name']==loser_name].loser_elo.iloc[i+1:]
    sub_lw = df[df['loser_name']==winner_name].loser_elo.iloc[i+1:]
    sub_wl = df[df['winner_name']==loser_name].winner_elo.iloc[i+1:]
    
    sub_ww.replace(to_replace=sub_ww.values.tolist(), value=np.around(elo_winner), inplace=True)
    sub_ll.replace(to_replace=sub_ll.values.tolist(), value=np.around(elo_loser), inplace=True)
    sub_lw.replace(to_replace=sub_lw.values.tolist(), value=np.around(elo_winner), inplace=True)
    sub_wl.replace(to_replace=sub_wl.values.tolist(), value=np.around(elo_loser), inplace=True)
    
    df.update(sub_ww)
    df.update(sub_ll)
    df.update(sub_lw)
    df.update(sub_wl)

In [320]:
df

,tourney_id,tourney_name,series,court,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,winner_elo,loser_elo
0,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,6,104571,...,1.377,3.410,1.400,2.750,NaN,NaN,NaN,NaN,1515.0000,1485.0000
1,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,30,103285,...,2.350,1.685,2.000,1.727,NaN,NaN,NaN,NaN,1515.0000,1485.0000
2,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,16,103285,...,1.087,9.950,1.030,10.000,NaN,NaN,NaN,NaN,1515.0000,1485.0000
3,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,24,103285,...,1.159,6.390,1.125,5.500,NaN,NaN,NaN,NaN,1529.3528,1485.6472
4,2010-339,Brisbane International,ATP250,Outdoor,Hard,32,A,2010-01-03,11,103722,...,1.483,2.900,1.440,2.630,NaN,NaN,NaN,NaN,1515.0000,1485.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,1,103819,...,1.140,7.100,1.130,6.000,1.14,8.20,1.11,6.39,1529.3528,1485.6472
2675,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,4,103819,...,1.420,3.180,1.360,3.000,1.50,3.24,1.40,2.92,1529.3528,1485.6472
2676,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,14,104925,...,1.480,2.920,1.440,2.750,1.48,3.15,1.40,2.88,1515.6472,1499.3528
2677,2010-605,ATP World Tour Finals,Masters Cup,Indoor,Hard,8,F,2010-11-21,13,104925,...,1.250,4.600,1.250,3.750,1.30,4.76,1.24,4.03,1514.3528,1470.6472


In [301]:
df.winner_elo.unique()

array([1515.        , 1529.35280001, 1543.08854424, 1500.64719999,
       1556.23774139, 1515.64719999, 1528.70800524, 1514.35280001,
       1516.26425577, 1516.85080285, 1501.29199476, 1514.40728902,
       1500.        , 1515.61925815, 1500.05458207, 1501.2096737 ,
       1501.85696675, 1529.97211242, 1499.97205816, 1514.38074185,
       1529.38165205, 1500.67508757, 1530.        , 1528.68027515,
       1513.73349208, 1514.32491243, 1515.02794184, 1516.26650792,
       1529.38069207, 1514.97211242, 1502.50205642, 1513.78798929,
       1514.97205816, 1568.83074765, 1529.32747514, 1503.03319316,
       1530.61925815])

In [302]:
df.loser_elo.unique()

array([1485.        , 1485.64719999, 1486.26425577, 1484.35280001,
       1486.85080285, 1499.35280001, 1471.29199476, 1470.64719999,
       1513.08854424, 1526.23774139, 1498.70800524, 1471.85696675,
       1470.        , 1485.02794184, 1471.2096737 , 1470.05458207,
       1484.40728902, 1485.67508757, 1469.38074185, 1484.97205816,
       1472.52980372, 1484.97211242, 1500.        , 1470.67252486,
       1470.61930793, 1470.02788758, 1485.61925815, 1499.38069207,
       1486.26650792, 1470.67508757, 1498.76219935, 1471.82906649,
       1484.38074185, 1487.40699373, 1471.31972485, 1538.20454823,
       1500.02794184])

In [288]:
sub.winner_elo.iloc[3:].replace(to_replace=sub.winner_elo.iloc[3:].values.tolist(), value=2600, inplace=True)

In [290]:
sub

,tourney_name,winner_name,winner_elo,loser_elo
167,Australian Open,Novak Djokovic,1515.0,1485.0
168,Australian Open,Novak Djokovic,1515.0,1485.0
169,Australian Open,Novak Djokovic,1515.0,1485.0
170,Australian Open,Novak Djokovic,2600.0,1485.0
457,ABN AMRO World Tennis Tournament,Novak Djokovic,2600.0,1485.0
458,ABN AMRO World Tennis Tournament,Novak Djokovic,2600.0,1485.0
459,ABN AMRO World Tennis Tournament,Novak Djokovic,2600.0,1485.0
627,Dubai Tennis Championships,Novak Djokovic,2600.0,1485.0
628,Dubai Tennis Championships,Novak Djokovic,2600.0,1485.0
629,Dubai Tennis Championships,Novak Djokovic,2600.0,1485.0
